## Libraries

### Installation

In [7]:
!pip install git+https://www.github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install bitsandbytes
!pip install einops
!pip install --upgrade torch torchvision
!pip install scikit-learn
!pip install matplotlib
!pip install datasets
!pip install Bio
!pip install pybedtools
!pip install tabulate

  Cloning https://www.github.com/huggingface/transformers to /tmp/pip-req-build-xga8q_sp
  Running command git clone --filter=blob:none --quiet https://www.github.com/huggingface/transformers /tmp/pip-req-build-xga8q_sp
  Resolved https://www.github.com/huggingface/transformers to commit c96aca3a8d66d64f868a3e3967be624d79213bef
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-vgmfnrvf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-vgmfnrvf
  Resolved https://github.com/huggingface/accelerate to commit abce3604f0de71cb947d6cf653c51b3e19f65162
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypro

In [1]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoModel
from transformers.models.bert.configuration_bert import BertConfig
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
from Bio import SeqIO
from pybedtools import BedTool

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn

/data/Dcode/gaetano/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Models

In [2]:
# model name from huggingface.co/model name_id:model_name
model_ckpt = 'InstaDeepAI/nucleotide-transformer-500m-1000g'

# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, trust_remote_code=True) 
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, trust_remote_code=True) 

/data/Dcode/gaetano/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-1000g and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Datasetes

### get fasta hg19 database

In [3]:
FASTA_FILE = "/data/Dcode/gaetano/repos/Transformers4Genomic/data/hg19.fa"

def get_chrom2seq(capitalize=True):

    chrom2seq = {}
    for seq in SeqIO.parse(FASTA_FILE, "fasta"):
        chrom2seq[seq.description.split()[0]] = seq.seq.upper() if capitalize else seq.seq

    return chrom2seq

chrom2seq = get_chrom2seq()

## Get HepG2 data

In [4]:
bios_id = 'BioS73'
path_bios = '/data/Dcode/gaetano/repos/Transformers4Genomic/data/enhancers/biosamples/'

pos_beds = list(BedTool(f'{path_bios}{bios_id}_positive_1kb.bed'))
ctrl_beds = list(BedTool(f'{path_bios}{bios_id}_control_1kb.bed'))

pos_list = []
ctrl_list = []
for chr, start, end  in pos_beds:
    pos_list.append(str(chrom2seq[chr][int(start):int(end)]))

for chr, start, end  in ctrl_beds:
    ctrl_list.append(str(chrom2seq[chr][int(start):int(end)]))

ctrl_list = ctrl_list[:len(pos_list)]

seq_data = []
seq_data.extend(pos_list)
seq_data.extend(ctrl_list)


labels_data = []
labels_data.extend([1 for _ in range(len(pos_list))])
labels_data.extend([0 for _ in range(len(ctrl_list))])

### Get raQTL Sequences


In [5]:
import os
import pandas as pd

# data path
dataset_path = "/data/Dcode/gaetano/repos/Transformers4Genomic/data/raQTL/hepg2.sign.id.LP190708.txt"
raqtl_df = pd.read_csv(dataset_path, sep='\t')

### Take a small sample for testing 
# 0.1 = 10%, 0.2 = 20%, .. , 1.00 = 100
sample_size = 0.1

# get bioS ids and factors
bios_ids = 'hepg2'

### get sequences from dataframe 

In [133]:
seq_ref = []
seq_alt = []
exp_ref = []
exp_alt = []

lenght_bp = 500

for _, row in raqtl_df.iterrows():
    chromosome = row['chr']
    abspos = row['SNPabspos']
    seq = str(chrom2seq[chromosome][(abspos - 1 - int(lenght_bp / 2)):abspos + int(lenght_bp / 2)])

    seq_ref.append(seq)
    seq_alt.append(f"{seq[:int(len(seq)/2)]}{row['alt']}{seq[int(len(seq)/2)+1:]}")
    exp_ref.append(row['hepg2.ref.mean'])
    exp_alt.append(row['hepg2.alt.mean'])

### Create dataframe and remove Ns seq

In [132]:

# Create DataFrame
#bioS = pd.DataFrame({'seq_ref':seq_ref, 'seq_alt':seq_alt,'label_exp_ref': np.array(exp_ref), 'label_exp_alt':np.array(exp_alt)})
#bioS = pd.DataFrame({'seq_data': seq_ref + seq_alt, 'labels': exp_ref + exp_alt})
bioS = pd.DataFrame({'seq_data': seq_data, 'labels': labels_data})

# Filter out rows with sequences consisting only of the same character (presumably Ns)
bioS_no_Ns = bioS[bioS['seq_data'].apply(lambda x: len(set(x)) > 1)]

# Describe the 'labels' column of bioS_no_Ns
bioS_no_Ns['labels'].value_counts()

labels
1    14062
0    12688
Name: count, dtype: int64

In [8]:
bioS_no_Ns_sampled = bioS_no_Ns.sample(round(len(bioS_no_Ns)*sample_size),random_state=10)
bioS_no_Ns_sampled['labels'].describe()

count    2675.000000
mean        0.518879
std         0.499737
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: labels, dtype: float64

### Split train/val/test

In [47]:
# Get training data
train_sequences_bioS = bioS_no_Ns_sampled['seq_data'].tolist()
train_labels_bioS = bioS_no_Ns_sampled['labels'].tolist()

# Split the dataset into a training and a validation dataset
train_sequences_bioS, test_sequences_bioS, train_labels_bioS, test_labels_bioS = train_test_split(train_sequences_bioS,
                                                                              train_labels_bioS, test_size=0.20, random_state=42)

# Split the test data into validation and test sets
validation_sequences_bioS, test_sequences_bioS, validation_labels_bioS, test_labels_bioS = train_test_split(test_sequences_bioS, test_labels_bioS, test_size=0.50, random_state=42)

ds_train_bioS = Dataset.from_dict({"data": train_sequences_bioS, "labels": train_labels_bioS})
ds_validation_bioS = Dataset.from_dict({"data": validation_sequences_bioS, "labels": validation_labels_bioS})
ds_test_bioS = Dataset.from_dict({"data": test_sequences_bioS, "labels": test_labels_bioS})


ds_raqtl_ref_bioS = Dataset.from_dict({"data": seq_ref, "labels": exp_ref})
ds_raqtl_alt_bioS = Dataset.from_dict({"data": seq_alt, "labels": exp_alt})

In [57]:
train_sequences_bioS

['...',
 ...]

### Tokenize the dataset

In [85]:
def tokenize_function_single_dataset(examples, tokenizer=tokenizer):
    # encoder
    encoding = tokenizer(examples['data'], truncation=True, max_length=512, padding=True, return_tensors="pt")
    encoding['labels'] = [str(1) for _ in range(len(examples['labels']))] 
    
    return encoding

def get_tokenized_dataset(tokenizer):
    
    def tokenize_function(examples):
        # encode sequences
        encoding = tokenizer(examples['data'], truncation=True, max_length=512, padding=True, return_tensors="pt")
        
        # encode labels
        encoding['labels'] = examples['labels']
        
        return encoding
    
    tokenized_datasets_train = ds_train_bioS.map(tokenize_function, batched=True, remove_columns=["data"])
    tokenized_datasets_validation = ds_validation_bioS.map(tokenize_function, batched=True, remove_columns=["data"])
    tokenized_datasets_test = ds_test_bioS.map(tokenize_function, batched=True, remove_columns=["data"])

    return tokenized_datasets_train, tokenized_datasets_validation, tokenized_datasets_test


In [86]:
tokenized_dataset_raqtl_ref = ds_raqtl_ref_bioS.map(tokenize_function_single_dataset, batched=True, remove_columns=["data"])
tokenized_dataset_raqtl_al = ds_raqtl_ref_bioS.map(tokenize_function_single_dataset, batched=True, remove_columns=["data"])

Map: 100%|██████████| 14183/14183 [00:06<00:00, 2197.37 examples/s]


In [11]:
# get tokenized data for train/valid/test
tokenized_datasets_train, tokenized_datasets_validation, tokenized_datasets_test = get_tokenized_dataset(tokenizer)


Map: 100%|██████████| 268/268 [00:00<00:00, 1732.79 examples/s]


## Training 

### Training Args

In [13]:
from datetime import datetime
from transformers import EarlyStoppingCallback, IntervalStrategy

# datetime object containing current date and time
now = datetime.now()

batch_size = 32
log_steps = 100

args_training = TrainingArguments(
    f'finetuned_output {now}', # Adjust output directory as needed
    per_device_train_batch_size=batch_size, # Adjust batch size as needed
    per_device_eval_batch_size=batch_size, # Adjust batch size as needed
    learning_rate=2e-5, # Adjust learning rate as needed
    num_train_epochs=3, # Adjust number of training epochs as needed
    logging_steps=log_steps, # Log every 100 steps
    logging_dir='./logs',  # Directory for storing logs
    evaluation_strategy="steps", # Evaluate after each epoch
    save_strategy="steps", # Save model after each epoch
    save_total_limit=3,  # Limit the number of saved models
    disable_tqdm=False,  # Disable tqdm progress bars
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="f1_score"  # Define the metric for selecting the best model
)


/data/Dcode/gaetano/venv/lib/python3.10/site-packages/transformers/training_args.py:1449: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Metrics

In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve
from transformers import EarlyStoppingCallback, IntervalStrategy


## Metric Regression
def compute_metrics_regression(eval_pred):
    predictions, labels = eval_pred
    mse = mean_squared_error(labels, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(labels, predictions)
    r_squared = r2_score(labels, predictions)
    
    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "r_squared": r_squared
    }



# Define the metric for the evaluation using the f1 score
def compute_metrics_classification_binary(eval_pred):
    """Computes F1 score for binary classification"""
    predictions = np.argmax(eval_pred.predictions, axis=-1)
    references = eval_pred.label_ids
    
    r={'f1_score': metrics.f1_score(references, predictions),
       'precision': metrics.precision_score(references, predictions),
       'recall': metrics.recall_score(references, predictions),
       'accuracy': metrics.accuracy_score(references, predictions),
       'mcc_score': metrics.matthews_corrcoef(references, predictions),
       'roc_auc_score': metrics.roc_auc_score(references, predictions),
       }
    
    return r


### Train

In [15]:
# Define the working device
device = torch.device("cuda")

# Trainer
trainer = Trainer(
    model=model.to(device),
    args=args_training,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_classification_binary
    )

# Train the model
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss


TrainOutput(global_step=201, training_loss=0.36009262932177205, metrics={'train_runtime': 214.5072, 'train_samples_per_second': 29.929, 'train_steps_per_second': 0.937, 'total_flos': 3121566701533560.0, 'train_loss': 0.36009262932177205, 'epoch': 3.0})

In [60]:
from tabulate import tabulate

# Evaluate the model
eval_results = trainer.evaluate(tokenized_datasets_test)

# Print evaluation results in a table format
print(tabulate(eval_results.items(), headers=["Metric", "Value"]))


Metric                       Value
-----------------------  ---------
eval_loss                 0.541386
eval_f1_score             0.829091
eval_precision            0.820144
eval_recall               0.838235
eval_accuracy             0.824627
eval_mcc_score            0.649221
eval_roc_auc_score        0.824421
eval_runtime              2.8423
eval_samples_per_second  94.29
eval_steps_per_second     3.166
epoch                     3


In [90]:
# Assuming tokenized_datasets_test is a Dataset object from Hugging Face's datasets library

# Define the number of samples you want to predict on
num_samples = 5  # Adjust this number as needed

# Slice the test dataset
small_test_dataset = tokenized_datasets_test.select(range(num_samples))

# Make predictions on the sliced dataset
predictions = trainer.predict(small_test_dataset)

# Print the predictions
print(predictions.predictions)

[[-1.2905777  1.824972 ]
 [-0.8442128  1.1318097]
 [ 3.8644996 -3.0965126]
 [-1.3427682  1.7988319]
 [-1.3408266  1.7421963]]


In [152]:
# 1. Prepare the input DNA sequence
idx = 3
dna_sequence = [seq_ref[idx], seq_alt[idx]]

# 2. Tokenize the DNA sequence
tokenized_sequence = tokenizer(dna_sequence, padding=True, return_tensors="pt")

# 3. Move the tokenized sequence to the same device as your model
tokenized_sequence = {key: value.to(trainer.args.device) for key, value in tokenized_sequence.items()}

# 4. Pass the tokenized sequence through your trained model
with torch.no_grad():
    outputs = trainer.model(**tokenized_sequence)

# 5. Interpret the model output (depending on your task)
# For example, if it's a classification task:
probabilities = torch.softmax(outputs.logits, dim=1)
predicted_class = torch.argmax(probabilities, dim=1)

# Print or use the predictions
print("Predicted class:", predicted_class)

Predicted class: tensor([0, 0], device='cuda:0')


In [155]:
dna_sequence

['CCAGGCTAGAGTGCAGTGGCACGAACTTGGCTCACTGCAACCTCCGACTCCCAGGTTCAAAGGATTCTCCCATCTCAGCCTCCTGAGTAGCTGGGATTAGAGGTGGCCACCACCACACCCAGCTAGTTTTTTTTGTACTTTTAGTAGAGACAGTGTTTCACTATGTTGGCCAGGCTGGTCTCGAATTCTCGGCCTCAAGTGATCTGCCTGCCTCGGCCTCCCAAAGTGCTGGGATTATGGGCATGAGCCACCATGCCCAGCCAGGCTTAAGTACATTTTGAACAGCTAGTTTAATCTGCAGATGTGGAACTGTACATCCATCTCTGTGTTTACGAACTCTTACCTATGTGTACAGTCCCTTATTCACAACTTTGAAATCCAAAACGGTCTCTAAAAACTCAGAGAAAGTTTTAAAAACTTTTCCAGCAACAAAACCTGAACTGAGATCAAGGTCTGATCAGAACTTATCTGAAGCTACTGATAGTCTTTATCTTCCTCCTT',
 'CCAGGCTAGAGTGCAGTGGCACGAACTTGGCTCACTGCAACCTCCGACTCCCAGGTTCAAAGGATTCTCCCATCTCAGCCTCCTGAGTAGCTGGGATTAGAGGTGGCCACCACCACACCCAGCTAGTTTTTTTTGTACTTTTAGTAGAGACAGTGTTTCACTATGTTGGCCAGGCTGGTCTCGAATTCTCGGCCTCAAGTGATCTGCCTGCCTCGGCCTCCCAAAGTGCTGGGATTATGGGCATGAGCCAACATGCCCAGCCAGGCTTAAGTACATTTTGAACAGCTAGTTTAATCTGCAGATGTGGAACTGTACATCCATCTCTGTGTTTACGAACTCTTACCTATGTGTACAGTCCCTTATTCACAACTTTGAAATCCAAAACGGTCTCTAAAAACTCAGAGAAAGTTTTAAAAACTTTTCCAGCAACAAAACCTGAACTGAGATCAAGGTCTGATCAGAACTTATCTGAAGCTACTGATAGTCTTTATC

In [153]:
tokenized_sequence

{'input_ids': tensor([[   3, 2626, 1233, 3643, 3632,  611, 2445, 3598, 1718, 1703, 3428,  969,
          2476, 1639, 2669, 3287, 2560, 1335, 3582,  654,  558,  919, 1369, 1494,
          1367, 1232, 2273, 1421,  475, 3731, 2554, 1733, 1651, 2660, 3530, 1963,
          2675, 2670,   59, 2560, 1315, 3616, 3726,  494,  935, 3668, 3365, 1396,
          2280, 3412, 1660, 3195, 3115, 1162, 2154, 1913, 1204, 2456, 2635, 1868,
          3501, 1116, 2089, 1797, 2564, 3037, 2308,  615,  775, 1364,   41, 1447,
          2084,  171,  160, 3172, 3947,  400,  597, 2503, 2347,  314, 1354, 1450,
          4102, 4101, 4101],
         [   3, 2626, 1233, 3643, 3632,  611, 2445, 3598, 1718, 1703, 3428,  969,
          2476, 1639, 2669, 3287, 2560, 1335, 3582,  654,  558,  919, 1369, 1494,
          1367, 1232, 2273, 1421,  475, 3731, 2554, 1733, 1651, 2660, 3530, 1963,
          2675, 2670,   59, 2560, 1315, 3616, 3718,  494,  935, 3668, 3365, 1396,
          2280, 3412, 1660, 3195, 3115, 1162, 2154, 1913

In [154]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.0783, -2.3712],
        [ 3.1306, -2.4272]], device='cuda:0'), hidden_states=None, attentions=None)